In [1]:
%%writefile matmul.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Kernel 定义
__global__ void matrix_multiplication_kernel(const float* A, const float* B, float* C, int M, int N, int K) {
    int row =  blockDim.y * blockIdx.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if(col >= K || row >= M){
        return;
    }
    
    float acc = 0.0f;
    for(int i = 0; i < N; i++){
        acc += A[row * N + i] * B[i * K + col];
    }
    C[row * K + col] = acc; 
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(16, 16);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_kernel<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Writing matmul.cu


上面的实现有一个很大的问题就是，在每一个线程里面，为了计算C里面的一个位置上面的值，需要去全局内存上访问col次数据A，也需要访问col次数据B，导致数据等待时间大大增加。解决方法在于，使用线程块里面的共享内存，在一个线程块里面，先读取A和B的一部分数据到线程块里面的共享内存里面，然后进行计算，接着读取下一部分。通过分块避免所有线程去读取全局内存里面的数据。

In [2]:
%%writefile matmul_v2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float* __restrict__ A, const float* __restrict__ B, float* C, int M, int N, int K) {
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // 计算当前线程在全局矩阵 C 中负责的坐标
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float acc = 0.0f;

    // 循环遍历所有的 Tile (以 TILE_WIDTH 为步长遍历 N 维度)
    for (int t = 0; t < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {

        // 1. 协作加载 A 的 Tile 到共享内存
        // 边界检查：防止索引越界 (Padding 0)
        if (row < M && t * TILE_WIDTH + tx < N) {
            As[ty][tx] = A[row * N + t * TILE_WIDTH + tx];
        } else {
            As[ty][tx] = 0.0f;
        }

        // 2. 协作加载 B 的 Tile 到共享内存
        if (col < K && t * TILE_WIDTH + ty < N) {
            // 注意这里 B 的索引：行是 t*TILE_WIDTH + ty, 列是 col
            Bs[ty][tx] = B[(t * TILE_WIDTH + ty) * K + col];
        } else {
            Bs[ty][tx] = 0.0f;
        }

        // 3. 必须同步！确保所有线程都加载完了数据
        __syncthreads();

        // 4. 在共享内存上进行计算
        for (int i = 0; i < TILE_WIDTH; ++i) {
            acc += As[ty][i] * Bs[i][tx];
        }

        // 5. 必须同步！确保在加载下一个 Tile 之前，当前 Tile 的数据已经用完
        __syncthreads();
    }

    // 写回结果
    if (row < M && col < K) {
        C[row * K + col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Writing matmul_v2.cu


In [3]:
%%writefile matmul_v3.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float *__restrict__ A, const float *__restrict__ B, float *C, int M, int N, int K)
{
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH+1];

    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int row = blockIdx.y * TILE_WIDTH + ty;
    int col = blockIdx.x * TILE_WIDTH + tx;

    float acc = 0.0f;

    for (int i = 0; i < (N + TILE_WIDTH - 1) / TILE_WIDTH; i++)
    {
        // 读取数据
        if (row < M && TILE_WIDTH * i + tx < N)
        {
            As[ty][tx] = A[row * N + TILE_WIDTH * i + tx];
        }
        else
        {
            As[ty][tx] = 0.0f;
        }
        if (col < K && TILE_WIDTH * i + ty < N)
        {
            Bs[ty][tx] = B[(TILE_WIDTH * i + ty) * K + col];
        }
        else
        {
            Bs[ty][tx] = 0.0f;
        }
        __syncthreads(); // 等待线程块里面的线程都搬运完成

        for (int j = 0; j < TILE_WIDTH; j++)
        {
            acc += As[ty][j] * Bs[j][tx]; //访问线程块里面的共享内存时，没有合并访问，只在乎bank config
        }

        __syncthreads();
    }
    if (row<M && col<K)
    {
        C[row*K+col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float *A, const float *B, float *C, int M, int N, int K)
{
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);

    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
    {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    // 等待 GPU 完成
    cudaDeviceSynchronize();
}


Writing matmul_v3.cu


In [4]:
%%writefile matmul_v4.cu
#include <stdio.h>
#include <cuda_runtime.h>

// 宏定义块大小
// TS (Tile Size): 每个 Block 计算 64x64 的 C
// WPT (Work Per Thread): 每个线程计算 4x4 的 C
// TS_K: K 维度(你的代码里是 N 维度)的分块大小，设为 8 或 16
#define TS 64
#define WPT 4
#define TS_K 16 

// 优化后的 Kernel
__global__ void matrix_multiplication_optimized(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 每个 Block 处理 C 中 TS x TS (64x64) 的区域
    // 线程块维度: dim3(TS/WPT, TS/WPT) -> (16, 16) -> 256 个线程
    
    // 1. 声明共享内存
    // As: 存储 A 的切片 [TS][TS_K] -> [64][16]
    // Bs: 存储 B 的切片 [TS_K][TS] -> [16][64]
    __shared__ float As[TS][TS_K];
    __shared__ float Bs[TS_K][TS];

    // 2. 声明寄存器
    // accum: 累加器，每个线程负责计算 4x4 = 16 个元素
    float accum[WPT][WPT] = {0.0f};
    
    // reg_A, reg_B: 用于在内循环中缓存从 SMEM 读取的值
    float reg_A[WPT];
    float reg_B[WPT];

    // 线程 ID 和 Block ID
    int tx = threadIdx.x; // range 0-15
    int ty = threadIdx.y; // range 0-15
    int bx = blockIdx.x;
    int by = blockIdx.y;

    // 当前线程负责的 C 矩阵起始坐标 (C 的分块左上角 + 线程偏移)
    // 每个线程覆盖 WPT(4) 个像素宽/高
    int row_c = by * TS + ty * WPT; 
    int col_c = bx * TS + tx * WPT;

    // 3. 循环遍历 N 维度 (步长 TS_K = 16)
    for (int t = 0; t < N; t += TS_K) {
        
        // --- 加载数据到 Shared Memory (协作加载) ---
        // 我们有 256 个线程。
        // 需要加载 A 的 Tile: 64行 * 16列 = 1024 元素。每个线程加载 1024/256 = 4 个元素。
        // 需要加载 B 的 Tile: 16行 * 64列 = 1024 元素。每个线程加载 4 个元素。
        
        // 加载 As (A 的子块): 
        // 这里的逻辑是将 256 个线程映射到 64x16 的区域
        // 我们使用 float4 向量化加载来极致优化带宽
        
        // 计算当前线程加载 As 的位置
        // 将 16x16 的线程块视为 256 个线性线程
        int tid = ty * (TS / WPT) + tx; // 0 ~ 255
        
        // 映射到 As[64][16]: 每一行 16 个元素，如果是 float4 就是 4 个 float4
        // 256 个线程，每个加载 1 个 float4 (4个float)，正好 1024 个 float
        // As 的行索引
        int load_a_row = tid / (TS_K / 4); 
        int load_a_col = (tid % (TS_K / 4)) * 4;
        
        // 从全局内存 A 加载到 As
        // 全局索引: A[(by * TS + load_a_row) * N + (t + load_a_col)]
        // 注意边界检查省略了，假设维度对其
        if (by * TS + load_a_row < M && t + load_a_col < N) {
             // 使用 float4 指针强转进行向量加载
             float4 tmp = reinterpret_cast<const float4*>(&A[(by * TS + load_a_row) * N + (t + load_a_col)])[0];
             As[load_a_row][load_a_col + 0] = tmp.x;
             As[load_a_row][load_a_col + 1] = tmp.y;
             As[load_a_row][load_a_col + 2] = tmp.z;
             As[load_a_row][load_a_col + 3] = tmp.w;
        }

        // 加载 Bs (B 的子块): [16][64]
        // 同样用 tid 映射。每行 64 个元素 = 16 个 float4。
        // 总共 16 行。总 float4 数 = 16 * 16 = 256。正好每个线程取 1 个 float4。
        int load_b_row = tid / (TS / 4);
        int load_b_col = (tid % (TS / 4)) * 4;

        if (t + load_b_row < N && bx * TS + load_b_col < K) {
             float4 tmp = reinterpret_cast<const float4*>(&B[(t + load_b_row) * K + (bx * TS + load_b_col)])[0];
             Bs[load_b_row][load_b_col + 0] = tmp.x;
             Bs[load_b_row][load_b_col + 1] = tmp.y;
             Bs[load_b_row][load_b_col + 2] = tmp.z;
             Bs[load_b_row][load_b_col + 3] = tmp.w;
        }

        __syncthreads(); // 等待数据加载完成

        // --- 在寄存器上进行计算 ---
        // 遍历 Shared Memory 中的 TS_K (16) 维度
        #pragma unroll
        for (int k = 0; k < TS_K; ++k) {
            
            // 1. 将所需的 As 和 Bs 数据预加载到寄存器
            // 每个线程计算 4x4，需要 As 的一列 4 个值，Bs 的一行 4 个值
            for (int i = 0; i < WPT; ++i) {
                reg_A[i] = As[ty * WPT + i][k];
                reg_B[i] = Bs[k][tx * WPT + i];
            }

            // 2. 外积计算 (Outer Product)
            // 计算 4x4 的结果，复用 reg_A 和 reg_B
            for (int row = 0; row < WPT; ++row) {
                for (int col = 0; col < WPT; ++col) {
                    accum[row][col] += reg_A[row] * reg_B[col];
                }
            }
        }
        
        __syncthreads(); // 等待计算完成，准备加载下一块
    }

    // 4. 写回结果到全局内存
    // 每个线程写回 4x4 个点
    for (int row = 0; row < WPT; ++row) {
        for (int col = 0; col < WPT; ++col) {
            int global_row = row_c + row;
            int global_col = col_c + col;
            
            if (global_row < M && global_col < K) {
                C[global_row * K + global_col] = accum[row][col];
            }
        }
    }
}

// Host 端调用示例
extern "C" void solve(const float* d_A, const float* d_B, float* d_C, int M, int N, int K) {
    // 线程块大小: 16x16 = 256 线程
    dim3 threadsPerBlock(TS / WPT, TS / WPT); 
    
    // Grid 大小: 因为每个 Block 处理 64x64，所以除以 TS(64)
    dim3 numBlocks((K + TS - 1) / TS, (M + TS - 1) / TS);

    matrix_multiplication_optimized<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, N, K);
}

Writing matmul_v4.cu


In [5]:
%%writefile matmul_v4_1.cu
#include <stdio.h>
#include <cuda_runtime.h>

// 宏定义块大小
// TS (Tile Size): 每个 Block 计算 64x64 的 C
// WPT (Work Per Thread): 每个线程计算 4x4 的 C
// TS_K: K 维度(你的代码里是 N 维度)的分块大小，设为 8 或 16
#define TS 64
#define WPT 4
#define TS_K 16 

// 优化后的 Kernel
__global__ void matrix_multiplication_optimized(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 每个 Block 处理 C 中 TS x TS (64x64) 的区域
    // 线程块维度: dim3(TS/WPT, TS/WPT) -> (16, 16) -> 256 个线程
    
    // 1. 声明共享内存
    // As: 存储 A 的切片 [TS][TS_K] -> [64][16]
    // Bs: 存储 B 的切片 [TS_K][TS] -> [16][64]
    __shared__ float As[TS][TS_K];
    __shared__ float Bs[TS_K][TS];

    // 2. 声明寄存器
    // accum: 累加器，每个线程负责计算 4x4 = 16 个元素
    float accum[WPT][WPT] = {0.0f};
    
    // reg_A, reg_B: 用于在内循环中缓存从 SMEM 读取的值
    float reg_A[WPT];
    float reg_B[WPT];

    // 线程 ID 和 Block ID
    int tx = threadIdx.x; // range 0-15
    int ty = threadIdx.y; // range 0-15
    int bx = blockIdx.x;
    int by = blockIdx.y;

    // 当前线程负责的 C 矩阵起始坐标 (C 的分块左上角 + 线程偏移)
    // 每个线程覆盖 WPT(4) 个像素宽/高
    int row_c = by * TS + ty * WPT; 
    int col_c = bx * TS + tx * WPT;

    // 3. 循环遍历 N 维度 (步长 TS_K = 16)
    for (int t = 0; t < N; t += TS_K) {
        
        // --- 加载数据到 Shared Memory (协作加载) ---
        // 我们有 256 个线程。
        // 需要加载 A 的 Tile: 64行 * 16列 = 1024 元素。每个线程加载 1024/256 = 4 个元素。
        // 需要加载 B 的 Tile: 16行 * 64列 = 1024 元素。每个线程加载 4 个元素。
        
        // 加载 As (A 的子块): 
        // 这里的逻辑是将 256 个线程映射到 64x16 的区域
        // 我们使用 float4 向量化加载来极致优化带宽
        
        // 计算当前线程加载 As 的位置
        // 将 16x16 的线程块视为 256 个线性线程
        int tid = ty * (TS / WPT) + tx; // 0 ~ 255
        
        // 映射到 As[64][16]: 每一行 16 个元素，如果是 float4 就是 4 个 float4
        // 256 个线程，每个加载 1 个 float4 (4个float)，正好 1024 个 float
        // As 的行索引
        int load_a_row = tid / (TS_K / 4); 
        int load_a_col = (tid % (TS_K / 4)) * 4;
        
        // 从全局内存 A 加载到 As
        // 全局索引: A[(by * TS + load_a_row) * N + (t + load_a_col)]
        // 注意边界检查省略了，假设维度对其
        if (by * TS + load_a_row < M && t + load_a_col < N) {
             // 使用 float4 指针强转进行向量加载
             float4 tmp = reinterpret_cast<const float4*>(&A[(by * TS + load_a_row) * N + (t + load_a_col)])[0];
             As[load_a_row][load_a_col + 0] = tmp.x;
             As[load_a_row][load_a_col + 1] = tmp.y;
             As[load_a_row][load_a_col + 2] = tmp.z;
             As[load_a_row][load_a_col + 3] = tmp.w;
        }

        // 加载 Bs (B 的子块): [16][64]
        // 同样用 tid 映射。每行 64 个元素 = 16 个 float4。
        // 总共 16 行。总 float4 数 = 16 * 16 = 256。正好每个线程取 1 个 float4。
        int load_b_row = tid / (TS / 4);
        int load_b_col = (tid % (TS / 4)) * 4;

        if (t + load_b_row < N && bx * TS + load_b_col < K) {
             float4 tmp = reinterpret_cast<const float4*>(&B[(t + load_b_row) * K + (bx * TS + load_b_col)])[0];
             Bs[load_b_row][load_b_col + 0] = tmp.x;
             Bs[load_b_row][load_b_col + 1] = tmp.y;
             Bs[load_b_row][load_b_col + 2] = tmp.z;
             Bs[load_b_row][load_b_col + 3] = tmp.w;
        }

        __syncthreads(); // 等待数据加载完成

        // --- 在寄存器上进行计算 ---
        // 遍历 Shared Memory 中的 TS_K (16) 维度
        #pragma unroll
        for (int k = 0; k < TS_K; ++k) {
            
            // 1. 将所需的 As 和 Bs 数据预加载到寄存器
            // 每个线程计算 4x4，需要 As 的一列 4 个值，Bs 的一行 4 个值
            for (int i = 0; i < WPT; ++i) {
                reg_A[i] = As[ty * WPT + i][k];
                reg_B[i] = Bs[k][tx * WPT + i];
            }

            // 2. 外积计算 (Outer Product)
            // 计算 4x4 的结果，复用 reg_A 和 reg_B
            for (int row = 0; row < WPT; ++row) {
                for (int col = 0; col < WPT; ++col) {
                    accum[row][col] += reg_A[row] * reg_B[col];
                }
            }
        }
        
        __syncthreads(); // 等待计算完成，准备加载下一块
    }

    // 4. 写回结果到全局内存
    // 每个线程写回 4x4 个点
    for (int row = 0; row < WPT; ++row) {
        for (int col = 0; col < WPT; ++col) {
            int global_row = row_c + row;
            int global_col = col_c + col;
            
            if (global_row < M && global_col < K) {
                C[global_row * K + global_col] = accum[row][col];
            }
        }
    }
}

// Host 端调用示例
extern "C" void solve(const float* d_A, const float* d_B, float* d_C, int M, int N, int K) {
    // 线程块大小: 16x16 = 256 线程
    dim3 threadsPerBlock(TS / WPT, TS / WPT); 
    
    // Grid 大小: 因为每个 Block 处理 64x64，所以除以 TS(64)
    dim3 numBlocks((K + TS - 1) / TS, (M + TS - 1) / TS);

    matrix_multiplication_optimized<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, N, K);
}

Writing matmul_v4_1.cu


In [6]:
%%writefile matmul_v5.cu
#include <stdio.h>
#include <cuda_runtime.h>

// ==========================================
// 针对 T4 (Compute Capability 7.5) 优化的参数
// ==========================================
// 每个 Block 计算 128x128 的 C
const int BM = 128;
const int BN = 128;
// K 维度的步进，每次加载 8 列
const int BK = 8;
// 每个线程计算 8x8 的 C
const int TM = 8;
const int TN = 8;

// 线程块大小: (BM/TM) * (BN/TN) = 16 * 16 = 256 线程
// 满足 T4 最佳 Occupancy

__global__ __launch_bounds__(256)
void sgemm_optimized_t4(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 线程索引
    int tid = threadIdx.x; // 0..255
    
    // 逻辑坐标 (16x16)
    int ty = tid / 16;
    int tx = tid % 16;

    // 当前 Block 负责的 C 的左上角坐标
    int by = blockIdx.y;
    int bx = blockIdx.x;
    
    // Shared Memory 声明
    // As: 存储 A 的切片 [BK][BM]。
    // 注意：这里我们将 A 转置存储 (Transposed)，为了后续计算时能向量化读取
    // Padding: 为了避免 Bank Conflicts，我们在行尾 +1 或 +4 (这里不需要，因为 128 是 32 的倍数，但读取方式不同)
    // 实际上，为了极致性能，我们让 As 的维度为 [BK][BM]，这样计算时按 As[k][row] 读取是连续的。
    __shared__ float As[BK][BM]; 
    __shared__ float Bs[BK][BN];

    // 寄存器累加器，8x8
    float accum[TM][TN] = {0.0f};

    // 用于从 SMEM 加载到寄存器的临时变量
    float rag[TM]; // 缓存 A 的一列
    float rbg[TN]; // 缓存 B 的一行

    // 计算当前 Block 在 Global Memory 中的起始位置
    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;
    float* C_ptr = C + by * BM * N + bx * BN;

    // ==========================================================
    // 预计算加载 Global Memory 的索引
    // 我们有 256 个线程。
    // A 的 Tile 是 128(Row) x 8(Col)。总共 1024 元素。每个线程搬运 4 个 (float4)。
    // B 的 Tile 是 8(Row) x 128(Col)。总共 1024 元素。每个线程搬运 4 个 (float4)。
    // ==========================================================

    // A 的加载索引 (为了转置存储到 SMEM)
    // 我们按 Global A 的行优先读取，写入到 As 的 [col][row]
    // tid 范围 0-255。
    // 128行 * 8列 / 4(float4) = 256 个 float4 操作。正好 1 线程 1 个 float4。
    // load_a_row: 0..127, load_a_col: 0, 4 (因为 K step 是 8)
    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;

    // B 的加载索引
    // 8行 * 128列 / 4 = 256 个 float4。
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    // 主循环：在 K 维度上推进
    for (int k_step = 0; k_step < K; k_step += BK) {
        
        // --------------------------------------------------------
        // 1. 加载数据到 Shared Memory (Vectorized Global Load)
        // --------------------------------------------------------
        
        // 加载 A (M x K): 使用 float4
        // 边界检查：假设 M, N, K 是 8/128 的倍数以获得最佳性能，这里加简单防护
        if (by * BM + load_a_row < M && k_step + load_a_col < K) {
            float4 tmp = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + k_step + load_a_col])[0];
            // 关键优化：转置写入 As
            // Global: A[row][k] -> SMEM: As[k][row]
            // 这样在计算阶段，同一个 k 的不同 row 是连续内存
            As[load_a_col + 0][load_a_row] = tmp.x;
            As[load_a_col + 1][load_a_row] = tmp.y;
            As[load_a_col + 2][load_a_row] = tmp.z;
            As[load_a_col + 3][load_a_row] = tmp.w;
        } else {
            // 边界 padding (设为0不影响累加)
             As[load_a_col + 0][load_a_row] = 0.0f;
             As[load_a_col + 1][load_a_row] = 0.0f;
             As[load_a_col + 2][load_a_row] = 0.0f;
             As[load_a_col + 3][load_a_row] = 0.0f;
        }

        // 加载 B (K x N): 使用 float4
        if (k_step + load_b_row < K && bx * BN + load_b_col < N) {
            float4 tmp = reinterpret_cast<const float4*>(&B_ptr[(k_step + load_b_row) * N + load_b_col])[0];
            // B 不需要转置，直接按行存
            reinterpret_cast<float4*>(&Bs[load_b_row][load_b_col])[0] = tmp;
        } else {
             Bs[load_b_row][load_b_col + 0] = 0.0f;
             Bs[load_b_row][load_b_col + 1] = 0.0f;
             Bs[load_b_row][load_b_col + 2] = 0.0f;
             Bs[load_b_row][load_b_col + 3] = 0.0f;
        }

        __syncthreads();

        // --------------------------------------------------------
        // 2. 计算 (Math Loop)
        // --------------------------------------------------------
        // 展开循环，计算 8 个 K 步骤
        #pragma unroll
        for (int k = 0; k < BK; ++k) {
            // 从 SMEM 加载 A 的一列 (TM=8) 到寄存器
            // 由于我们转置了 As，现在 As[k][row...] 是连续的！
            // 我们可以用 float4 加载来加速！
            // 当前线程计算的 C 的行是: ty * TM 到 ty * TM + 7
            // 对应的 SMEM 地址是: &As[k][ty * TM]
            float4 tmpA0 = reinterpret_cast<const float4*>(&As[k][ty * TM])[0];
            float4 tmpA1 = reinterpret_cast<const float4*>(&As[k][ty * TM + 4])[0];
            
            rag[0] = tmpA0.x; rag[1] = tmpA0.y; rag[2] = tmpA0.z; rag[3] = tmpA0.w;
            rag[4] = tmpA1.x; rag[5] = tmpA1.y; rag[6] = tmpA1.z; rag[7] = tmpA1.w;

            // 从 SMEM 加载 B 的一行 (TN=8) 到寄存器
            // 当前线程计算的 C 的列是: tx * TN 到 tx * TN + 7
            // 对应的 SMEM 地址是: &Bs[k][tx * TN]
            float4 tmpB0 = reinterpret_cast<const float4*>(&Bs[k][tx * TN])[0];
            float4 tmpB1 = reinterpret_cast<const float4*>(&Bs[k][tx * TN + 4])[0];

            rbg[0] = tmpB0.x; rbg[1] = tmpB0.y; rbg[2] = tmpB0.z; rbg[3] = tmpB0.w;
            rbg[4] = tmpB1.x; rbg[5] = tmpB1.y; rbg[6] = tmpB1.z; rbg[7] = tmpB1.w;

            // 外积 (Outer Product) 计算 8x8
            // 编译器会自动优化为 FFMA 指令
            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    accum[r][c] += rag[r] * rbg[c];
                }
            }
        }

        __syncthreads();
    }

    // --------------------------------------------------------
    // 3. 写回 Global Memory
    // --------------------------------------------------------
    // 每个线程负责 8x8 个点。
    // 为了带宽优化，我们也应该用 float4 写回。
    // 这稍微复杂一点，因为 thread 的 8x8 是块状的，不是完全连续的行。
    // 每个线程有 8 行，每行 8 个元素。每行的 8 个元素是连续的。
    // 可以用 2 个 float4 写回一行。

    int global_row_start = by * BM + ty * TM;
    int global_col_start = bx * BN + tx * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        int global_r = global_row_start + r;
        if (global_r < M) {
            int global_c = global_col_start;
            if (global_c + 7 < N) {
                // 常见的路径：可以直接 float4 写回
                float4 tmp0;
                tmp0.x = accum[r][0]; tmp0.y = accum[r][1]; tmp0.z = accum[r][2]; tmp0.w = accum[r][3];
                reinterpret_cast<float4*>(&C[global_r * N + global_c])[0] = tmp0;

                float4 tmp1;
                tmp1.x = accum[r][4]; tmp1.y = accum[r][5]; tmp1.z = accum[r][6]; tmp1.w = accum[r][7];
                reinterpret_cast<float4*>(&C[global_r * N + global_c + 4])[0] = tmp1;
            } else {
                // 边界处理
                for (int c = 0; c < TN; ++c) {
                    if (global_c + c < N) {
                        C[global_r * N + global_c + c] = accum[r][c];
                    }
                }
            }
        }
    }
}

// 主机端调用 Wrapper
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
    
    sgemm_optimized_t4<<<grid, block>>>(A, B, C, M, N, K);
}

// 测试 main 函数 (可选)
int main() {
    int M = 4096;
    int N = 4096;
    int K = 4096;
    size_t size = M * K * sizeof(float); // 简化测试，假设方阵

    float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
    
    // 分配内存... (略去分配和初始化代码以保持简洁)
    // cudaMalloc(&d_A, ...);
    
    // 调用
    // launch_sgemm_optimized(d_A, d_B, d_C, M, N, K);
    
    // cudaDeviceSynchronize();
    
    printf("Kernel compiled and structure ready for T4.\n");
    return 0;
}

Writing matmul_v5.cu


In [18]:
%%writefile matmul_v6.cu
// 第6个版本
#include <cuda_runtime.h>
#include <stdio.h>

// -------------------------------------------------------------------------
// 参数定义 (针对 T4 优化)
// -------------------------------------------------------------------------
const int BM = 128;
const int BN = 128;
const int BK = 8;
const int TM = 8;
const int TN = 8;

__global__ __launch_bounds__(256)
void sgemm_optimized_t4_kernel(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    int tid = threadIdx.x;
    int ty = tid / 16;
    int tx = tid % 16;
    int by = blockIdx.y;
    int bx = blockIdx.x;
    
    __shared__ float As[BK][BM]; 
    __shared__ float Bs[BK][BN];

    float accum[TM][TN] = {0.0f};
    float rag[TM];
    float rbg[TN];

    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;
    // C 指针后面计算

    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    for (int k_step = 0; k_step < K; k_step += BK) {
        // Load A (Transposed)
        if (by * BM + load_a_row < M && k_step + load_a_col < K) {
            float4 tmp = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + k_step + load_a_col])[0];
            As[load_a_col + 0][load_a_row] = tmp.x;
            As[load_a_col + 1][load_a_row] = tmp.y;
            As[load_a_col + 2][load_a_row] = tmp.z;
            As[load_a_col + 3][load_a_row] = tmp.w;
        } else {
             As[load_a_col + 0][load_a_row] = 0.0f;
             As[load_a_col + 1][load_a_row] = 0.0f;
             As[load_a_col + 2][load_a_row] = 0.0f;
             As[load_a_col + 3][load_a_row] = 0.0f;
        }

        // Load B
        if (k_step + load_b_row < K && bx * BN + load_b_col < N) {
            float4 tmp = reinterpret_cast<const float4*>(&B_ptr[(k_step + load_b_row) * N + load_b_col])[0];
            reinterpret_cast<float4*>(&Bs[load_b_row][load_b_col])[0] = tmp;
        } else {
             Bs[load_b_row][load_b_col + 0] = 0.0f;
             Bs[load_b_row][load_b_col + 1] = 0.0f;
             Bs[load_b_row][load_b_col + 2] = 0.0f;
             Bs[load_b_row][load_b_col + 3] = 0.0f;
        }

        __syncthreads();

        #pragma unroll
        for (int k = 0; k < BK; ++k) {
            float4 tmpA0 = reinterpret_cast<const float4*>(&As[k][ty * TM])[0];
            float4 tmpA1 = reinterpret_cast<const float4*>(&As[k][ty * TM + 4])[0];
            rag[0] = tmpA0.x; rag[1] = tmpA0.y; rag[2] = tmpA0.z; rag[3] = tmpA0.w;
            rag[4] = tmpA1.x; rag[5] = tmpA1.y; rag[6] = tmpA1.z; rag[7] = tmpA1.w;

            float4 tmpB0 = reinterpret_cast<const float4*>(&Bs[k][tx * TN])[0];
            float4 tmpB1 = reinterpret_cast<const float4*>(&Bs[k][tx * TN + 4])[0];
            rbg[0] = tmpB0.x; rbg[1] = tmpB0.y; rbg[2] = tmpB0.z; rbg[3] = tmpB0.w;
            rbg[4] = tmpB1.x; rbg[5] = tmpB1.y; rbg[6] = tmpB1.z; rbg[7] = tmpB1.w;

            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    accum[r][c] += rag[r] * rbg[c];
                }
            }
        }
        __syncthreads();
    }

    // Write back
    int global_row_start = by * BM + ty * TM;
    int global_col_start = bx * BN + tx * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        int global_r = global_row_start + r;
        if (global_r < M) {
            int global_c = global_col_start;
            if (global_c + 7 < N) {
                float4 tmp0, tmp1;
                tmp0.x = accum[r][0]; tmp0.y = accum[r][1]; tmp0.z = accum[r][2]; tmp0.w = accum[r][3];
                tmp1.x = accum[r][4]; tmp1.y = accum[r][5]; tmp1.z = accum[r][6]; tmp1.w = accum[r][7];
                reinterpret_cast<float4*>(&C[global_r * N + global_c])[0] = tmp0;
                reinterpret_cast<float4*>(&C[global_r * N + global_c + 4])[0] = tmp1;
            } else {
                for (int c = 0; c < TN; ++c) {
                    if (global_c + c < N) {
                        C[global_r * N + global_c + c] = accum[r][c];
                    }
                }
            }
        }
    }
}

// C++ 调用接口
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
    sgemm_optimized_t4_kernel<<<grid, block>>>(A, B, C, M, N, K);
}

Overwriting matmul_v6.cu


In [4]:
%%writefile matmul_v7.cu
// 第7个版本
#include <cuda_runtime.h>
#include <stdio.h>

// ==========================================
// T4 优化参数 + 双重缓冲策略
// ==========================================
const int BM = 128;
const int BN = 128;
const int BK = 8;
const int TM = 8;
const int TN = 8;

__global__ __launch_bounds__(256)
void sgemm_double_buffer_t4(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    int tid = threadIdx.x;
    int ty = tid / 16;
    int tx = tid % 16;
    int by = blockIdx.y;
    int bx = blockIdx.x;
    
    // =========================================================
    // 关键改变 1: 双重缓冲 Shared Memory
    // 使用 [2] 个 buffer，write_stage 用于写，read_stage 用于算
    // =========================================================
    __shared__ float As[2][BK][BM]; 
    __shared__ float Bs[2][BK][BN];

    float accum[TM][TN] = {0.0f};
    
    // 寄存器缓存，用于计算
    float rag[TM];
    float rbg[TN];

    // 寄存器缓存，用于 Global Memory 预取 (Prefetch)
    // 每个线程搬运 1 个 float4 的 A 和 1 个 float4 的 B
    float4 load_a_reg; 
    float4 load_b_reg;

    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;

    // 加载索引计算
    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    // =========================================================
    // Prologue (序幕): 加载第一个 Tile 到 Buffer 0
    // =========================================================
    {
        int k_start = 0;
        // Load A
        if (by * BM + load_a_row < M && k_start + load_a_col < K) {
            load_a_reg = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + k_start + load_a_col])[0];
        } else {
            load_a_reg = {0.0f, 0.0f, 0.0f, 0.0f};
        }
        // Load B
        if (k_start + load_b_row < K && bx * BN + load_b_col < N) {
            load_b_reg = reinterpret_cast<const float4*>(&B_ptr[(k_start + load_b_row) * N + load_b_col])[0];
        } else {
            load_b_reg = {0.0f, 0.0f, 0.0f, 0.0f};
        }

        // 写入 SMEM Buffer 0
        // A 转置写入
        As[0][load_a_col + 0][load_a_row] = load_a_reg.x;
        As[0][load_a_col + 1][load_a_row] = load_a_reg.y;
        As[0][load_a_col + 2][load_a_row] = load_a_reg.z;
        As[0][load_a_col + 3][load_a_row] = load_a_reg.w;
        
        // B 直接写入
        reinterpret_cast<float4*>(&Bs[0][load_b_row][load_b_col])[0] = load_b_reg;
    }

    __syncthreads();

    // =========================================================
    // Main Loop
    // =========================================================
    int write_stage_idx = 1; // 下一轮写入的位置
    int read_stage_idx = 0;  // 当前计算读取的位置

    // 注意：循环从 k=0 开始算，但在 k 时我们要预加载 k+BK 的数据
    for (int k = 0; k < K; k += BK) {
        
        // -----------------------------------------------------
        // 1. Prefetch Next Tile to Registers (Global -> Register)
        // 这里的关键是：当我们发起 Global Load 指令后，GPU 不会阻塞，
        // 而是会继续向下执行计算指令 (Math)，从而隐藏内存延迟。
        // -----------------------------------------------------
        int next_k = k + BK;
        if (next_k < K) {
            // Load A to Reg
            if (by * BM + load_a_row < M && next_k + load_a_col < K) {
                load_a_reg = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + next_k + load_a_col])[0];
            } else {
                load_a_reg = {0.0f, 0.0f, 0.0f, 0.0f};
            }
            // Load B to Reg
            if (next_k + load_b_row < K && bx * BN + load_b_col < N) {
                load_b_reg = reinterpret_cast<const float4*>(&B_ptr[(next_k + load_b_row) * N + load_b_col])[0];
            } else {
                load_b_reg = {0.0f, 0.0f, 0.0f, 0.0f};
            }
        }

        // -----------------------------------------------------
        // 2. Compute Current Tile (Register <-> SMEM)
        // 使用 read_stage_idx
        // -----------------------------------------------------
        #pragma unroll
        for (int i = 0; i < BK; ++i) {
            // Load A from SMEM to Reg
            float4 tmpA0 = reinterpret_cast<const float4*>(&As[read_stage_idx][i][ty * TM])[0];
            float4 tmpA1 = reinterpret_cast<const float4*>(&As[read_stage_idx][i][ty * TM + 4])[0];
            rag[0] = tmpA0.x; rag[1] = tmpA0.y; rag[2] = tmpA0.z; rag[3] = tmpA0.w;
            rag[4] = tmpA1.x; rag[5] = tmpA1.y; rag[6] = tmpA1.z; rag[7] = tmpA1.w;

            // Load B from SMEM to Reg
            float4 tmpB0 = reinterpret_cast<const float4*>(&Bs[read_stage_idx][i][tx * TN])[0];
            float4 tmpB1 = reinterpret_cast<const float4*>(&Bs[read_stage_idx][i][tx * TN + 4])[0];
            rbg[0] = tmpB0.x; rbg[1] = tmpB0.y; rbg[2] = tmpB0.z; rbg[3] = tmpB0.w;
            rbg[4] = tmpB1.x; rbg[5] = tmpB1.y; rbg[6] = tmpB1.z; rbg[7] = tmpB1.w;

            // Compute
            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    accum[r][c] += rag[r] * rbg[c];
                }
            }
        }

        // -----------------------------------------------------
        // 3. Store Prefetched Data to SMEM (Register -> SMEM)
        // 此时计算已经完成，我们等待所有线程都算完了当前块
        // -----------------------------------------------------
        __syncthreads(); // 确保 read_stage 的数据大家都不用了

        if (next_k < K) {
            // 将寄存器里的下一块数据写入 write_stage 的 SMEM
            // A Transposed
            As[write_stage_idx][load_a_col + 0][load_a_row] = load_a_reg.x;
            As[write_stage_idx][load_a_col + 1][load_a_row] = load_a_reg.y;
            As[write_stage_idx][load_a_col + 2][load_a_row] = load_a_reg.z;
            As[write_stage_idx][load_a_col + 3][load_a_row] = load_a_reg.w;

            // B Direct
            reinterpret_cast<float4*>(&Bs[write_stage_idx][load_b_row][load_b_col])[0] = load_b_reg;
        }

        // 翻转 buffer 索引
        // write_stage: 1 -> 0 -> 1
        // read_stage:  0 -> 1 -> 0
        write_stage_idx ^= 1;
        read_stage_idx ^= 1;

        __syncthreads(); // 确保 write_stage 的数据已经写好，可以作为下一轮的 read_stage
    }

    // =========================================================
    // Write Back
    // =========================================================
    int global_row_start = by * BM + ty * TM;
    int global_col_start = bx * BN + tx * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        int global_r = global_row_start + r;
        if (global_r < M) {
            int global_c = global_col_start;
            if (global_c + 7 < N) {
                float4 tmp0, tmp1;
                tmp0.x = accum[r][0]; tmp0.y = accum[r][1]; tmp0.z = accum[r][2]; tmp0.w = accum[r][3];
                tmp1.x = accum[r][4]; tmp1.y = accum[r][5]; tmp1.z = accum[r][6]; tmp1.w = accum[r][7];
                reinterpret_cast<float4*>(&C[global_r * N + global_c])[0] = tmp0;
                reinterpret_cast<float4*>(&C[global_r * N + global_c + 4])[0] = tmp1;
            } else {
                for (int c = 0; c < TN; ++c) {
                    if (global_c + c < N) {
                        C[global_r * N + global_c + c] = accum[r][c];
                    }
                }
            }
        }
    }
}

// 宿主调用
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
    sgemm_double_buffer_t4<<<grid, block>>>(A, B, C, M, N, K);
}

Overwriting matmul_v7.cu


In [1]:
%%writefile matmul_v8.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <chrono>
#include <random>

// ==========================================
// 核心超参数配置 (针对 T4 调优)
// ==========================================
const int BM = 128; // Block size M
const int BN = 128; // Block size N
const int BK = 8;   // K dimension step (K-blocking)
const int TM = 8;   // Thread tile M (每个线程计算的行数)
const int TN = 8;   // Thread tile N (每个线程计算的列数)

// 检查参数是否合理：Block内的线程数 = (BM * BN) / (TM * TN)
// 这里: (128*128) / (8*8) = 16384 / 64 = 256 线程/Block
// 这是一个非常经典的配置

// ==========================================
// 辅助宏与函数
// ==========================================
#define CHECK_CUDA(func)                                                       \
    {                                                                          \
        cudaError_t status = (func);                                           \
        if (status != cudaSuccess) {                                           \
            std::cerr << "CUDA Error: " << cudaGetErrorString(status) << " at line " << __LINE__ << std::endl; \
            exit(EXIT_FAILURE);                                                \
        }                                                                      \
    }

// ==========================================
// 极致优化的 SGEMM Kernel
// ==========================================
__global__ void sgemm_optimized_v1(int M, int N, int K, float alpha, const float *A, const float *B, float beta, float *C) {
    // 线程索引
    const int tx = threadIdx.x;
    const int ty = threadIdx.y; // 本实现是一维blockDim，ty通常为0，但在逻辑上我们视为一维索引
    const int tid = threadIdx.x; // 0-255

    // Block 索引
    const int bx = blockIdx.x;
    const int by = blockIdx.y;

    // Shared Memory 分配
    // 使用 float4 向量化加载，所以不需要为了 float4 特意填充，但为了避免 bank conflict 可能需要 padding
    // A tile: BM x BK. B tile: BK x BN.
    __shared__ float s_a[BK][BM]; // 转置存储 A，方便后续读取
    __shared__ float s_b[BK][BN]; 

    // 寄存器文件 (每个线程的累加器)
    float r_c[TM][TN] = {0.0f};
    
    // 寄存器缓存，用于从 smem 读取操作数，减少 smem 压力
    float r_a[TM];
    float r_b[TN];

    // 计算当前 Block 需要处理的全局内存 A 和 B 的起始位置
    // A: 行由 by 决定 -> by * 128
    // B: 列由 bx 决定 -> bx * 128
    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;
    float* C_ptr = C + (by * BM * N + bx * BN);

    // -------------------------------------------------------
    // 向量化加载计算逻辑
    // -------------------------------------------------------
    // 每个 Block 有 256 个线程。
    // 加载 A (BM x BK = 128 x 8 = 1024 floats). 256 线程，每个线程加载 4 个 float.
    // 正好一个 float4。
    
    // A 的加载索引计算:
    // 我们想把 A[Row][Col] 加载到 s_a[Col][Row] (转置) 或者 s_a[Row][Col]
    // 为了外积计算方便，我们通常希望 A 是列优先或转置的，B 是行优先的
    
    // 线程负责加载 A 的位置:
    // tid 0-255. 
    // load_a_row = tid / (BK/4)  (但BK=8, BK/4=2) -> tid / 2
    // load_a_col = (tid % 2) * 4
    // 这种映射比较复杂，我们需要更直观的映射。
    
    // 简化：我们将 A 视为 [BM, BK] 的矩阵。
    // 线程 k 加载 float4 from A memory.
    // 每一个线程加载一个 float4。
    // A_tile 共有 128行 * 8列。
    // 如果按行加载：每行需要 2个 float4 (8 floats)。
    // 128行 * 2 = 256 float4。正好对应 256 个线程。
    
    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;

    // B 的加载索引计算:
    // B_tile 共有 8行 * 128列。
    // 每行 128 floats = 32 个 float4。
    // 8行 * 32 = 256 float4。正好对应 256 个线程。
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    // -------------------------------------------------------
    // 主循环：在 K 维度上步进
    // -------------------------------------------------------
    for (int k = 0; k < K; k += BK) {
        // 1. 从 Global Memory 加载数据到 Shared Memory (向量化加载)
        
        // 加载 A
        float4 tmp_a = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + load_a_col + k])[0];
        // 存入 Shared Memory (注意转置，把 A 转置存，有利于计算时避免 Bank Conflict)
        // s_a[col][row]
        s_a[load_a_col + 0][load_a_row] = tmp_a.x;
        s_a[load_a_col + 1][load_a_row] = tmp_a.y;
        s_a[load_a_col + 2][load_a_row] = tmp_a.z;
        s_a[load_a_col + 3][load_a_row] = tmp_a.w;

        // 加载 B
        float4 tmp_b = reinterpret_cast<const float4*>(&B_ptr[load_b_row * N + load_b_col])[0];
        // 存入 Shared Memory
        // B 不需要转置，直接存 [row][col]
        reinterpret_cast<float4*>(&s_b[load_b_row][load_b_col])[0] = tmp_b;

        // 等待所有线程加载完毕
        __syncthreads();

        // 2. 计算 (核心 Math Loop)
        // BK = 8，循环 8 次
        #pragma unroll
        for (int i = 0; i < BK; ++i) {
            // 将 SMEM 数据读入寄存器
            
            // 线程计算位置确定:
            // 256个线程视作 16x16 的网格。
            // Block总大小 128x128。
            // 每个线程负责 8x8 的区域 (TMxTN)。
            // thread_row = (tid / 16) * 8
            // thread_col = (tid % 16) * 8
            
            int thread_row_start = (tid / 16) * TM;
            int thread_col_start = (tid % 16) * TN;

            // 加载 A 的一列 (长度 TM=8) 到寄存器
            // 由于 s_a 是转置存储的 [BK][BM]，所以这里取 s_a[i][row] 实际上是内存连续的
            // 这就是为什么要转置 A 的原因 -> 广播机制
            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                r_a[r] = s_a[i][thread_row_start + r];
            }

            // 加载 B 的一行 (长度 TN=8) 到寄存器
            #pragma unroll
            for (int c = 0; c < TN; ++c) {
                r_b[c] = s_b[i][thread_col_start + c];
            }

            // 外积计算 (Outer Product)
            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    r_c[r][c] += r_a[r] * r_b[c]; // FMA 指令
                }
            }
        }

        // 同步，准备加载下一块
        __syncthreads();
        
        // 移动 B 的指针 (A 的指针在索引里加 k 处理了，这里不需要动，或者需要根据逻辑调整)
        // 其实这里 A_ptr 应该是不动的，因为我们在索引里用了 +k
        // 但是 B_ptr 需要移动到下一块 K
        B_ptr += BK * N; 
        // A_ptr 其实不需要动，因为上面是 load_a_row * K + k。如果这里不动，上面就要 k += BK
    }

    // 3. 将结果写回 Global Memory
    // 同样需要向量化写回 (如果可能)
    int thread_row_start = (tid / 16) * TM;
    int thread_col_start = (tid % 16) * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        #pragma unroll
        for (int c = 0; c < TN; c += 4) {
            // 计算全局坐标
            int global_row = by * BM + thread_row_start + r;
            int global_col = bx * BN + thread_col_start + c;
            
            // 这里的写入由于 C 的布局是行优先，
            // 线程 (r, c) 处理的连续 4 个 float 正好是内存连续的，可以用 float4 写入
            
            float4 val;
            val.x = alpha * r_c[r][c + 0] + beta * C_ptr[(thread_row_start + r) * N + thread_col_start + c + 0];
            val.y = alpha * r_c[r][c + 1] + beta * C_ptr[(thread_row_start + r) * N + thread_col_start + c + 1];
            val.z = alpha * r_c[r][c + 2] + beta * C_ptr[(thread_row_start + r) * N + thread_col_start + c + 2];
            val.w = alpha * r_c[r][c + 3] + beta * C_ptr[(thread_row_start + r) * N + thread_col_start + c + 3];

            // 写入
            reinterpret_cast<float4*>(&C[global_row * N + global_col])[0] = val;
        }
    }
}

// 宿主调用
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid(N / BN, M / BM);

    // Warmup
    sgemm_optimized_v1<<<grid, block>>>(M, N, K, 1.0f, A, B, 0.0f, C);
}

Overwriting matmul_v8.cu


In [2]:
import os
import subprocess
for v in os.listdir(os.path.abspath('.')):
    prefix, end = os.path.splitext(v)
    if end == '.cu':
        subprocess.run(f"nvcc -shared -o {prefix}.so {prefix}.cu -Xcompiler -fPIC", shell=True)
print(os.listdir())

matmul_v8.cu(37): warning #177-D: variable "tx" was declared but never referenced
      const int tx = threadIdx.x;
                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

matmul_v8.cu(38): warning #177-D: variable "ty" was declared but never referenced
      const int ty = threadIdx.y;
                ^

matmul_v5.cu(55): warning #177-D: variable "C_ptr" was declared but never referenced
      float* C_ptr = C + by * BM * N + bx * BN;
             ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

matmul_v5.cu(206): warning #177-D: variable "N" was declared but never referenced
      int N = 4096;
          ^

matmul_v5.cu(208): warning #177-D: variable "size" was declared but never referenced
      size_t size = M * K * sizeof(float);
             ^

matmul_v5.cu(210): warning #177-D: variable "h_A" was declared but never referenced
      float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
             ^

matmul_v5.cu(210

['matmul_v3.so', 'matmul_v6.cu', 'matmul_v5.so', 'matmul_v8.cu', 'matmul_v5.cu', 'matmul_v2.cu', 'matmul_v7.cu', 'matmul_v3.cu', 'matmul_v6.so', 'matmul_v4_1.cu', 'matmul_v7.so', 'matmul.cu', 'build', '.virtual_documents', 'setup.py', 'matmul_v8.so', 'matmul_v4.so', 'matmul.so', 'matmul_v4_1.so', 'matmul_v2.so', 'matmul_v4.cu']


In [4]:
import torch
import ctypes
import numpy as np
import time

# --- 测试部分 ---

M, N, K = 1024, 1024, 1024

print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

device = torch.device("cuda")
A = torch.randn(M, N, device=device, dtype=torch.float32)
B = torch.randn(N, K, device=device, dtype=torch.float32)

C_torch = torch.matmul(A, B)

print("Pytorch: ")
%timeit torch.matmul(A, B); torch.cuda.synchronize()
print()

for v in sorted(os.listdir('.')):
    prefix, end = os.path.splitext(v)
    
    if end != '.so':
        continue

    lib = ctypes.CDLL(f'./{v}')

    # void solve(const float* A, const float* B, float* C, int M, int N, int K)
    # 指针对应 c_void_p, int 对应 c_int
    lib.solve.argtypes = [
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_int, 
        ctypes.c_int, 
        ctypes.c_int
    ]

    def cuda_matmul(a_tensor, b_tensor):
        M, N = a_tensor.shape
        N_b, K = b_tensor.shape
        
        assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
        
        c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
        
        if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
        if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
        
        lib.solve(
            ctypes.c_void_p(a_tensor.data_ptr()),
            ctypes.c_void_p(b_tensor.data_ptr()),
            ctypes.c_void_p(c_tensor.data_ptr()),
            ctypes.c_int(M),
            ctypes.c_int(N),
            ctypes.c_int(K)
        )
        
        return c_tensor

    C_custom = cuda_matmul(A, B)
    print(f"{v}")
    if torch.allclose(C_custom, C_torch, atol=1e-3):
        print(f"✅ 测试通过！结果正确。")
        %timeit cuda_matmul(A, B); torch.cuda.synchronize()
    else:
        print("❌ 测试失败。结果不一致。")
        print("最大误差:", (C_custom - C_torch).abs().max().item())
    print()

正在进行矩阵乘法测试: [1024x1024] * [1024x1024]
Pytorch: 
591 µs ± 2.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul.so
✅ 测试通过！结果正确。
4.38 ms ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v2.so
✅ 测试通过！结果正确。
2.42 ms ± 4.57 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v3.so
✅ 测试通过！结果正确。
2.4 ms ± 9.49 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v4.so
✅ 测试通过！结果正确。
783 µs ± 4.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v4_1.so
✅ 测试通过！结果正确。
790 µs ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v5.so
✅ 测试通过！结果正确。
694 µs ± 849 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v6.so
✅ 测试通过！结果正确。
696 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v7.so
✅ 测试通过！结果正确。
711 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v8.so
✅ 测试通过！结果正确。
722 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 1000 loop

In [5]:
import torch
import ctypes
import numpy as np
import time

# --- 测试部分 ---

M, N, K = 2048, 2048, 2048

print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

device = torch.device("cuda")
A = torch.randn(M, N, device=device, dtype=torch.float32)
B = torch.randn(N, K, device=device, dtype=torch.float32)

C_torch = torch.matmul(A, B)

print("Pytorch: ")
%timeit torch.matmul(A, B); torch.cuda.synchronize()
print()

for v in sorted(os.listdir('.')):
    prefix, end = os.path.splitext(v)
    
    if end != '.so':
        continue

    lib = ctypes.CDLL(f'./{v}')

    # void solve(const float* A, const float* B, float* C, int M, int N, int K)
    # 指针对应 c_void_p, int 对应 c_int
    lib.solve.argtypes = [
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_int, 
        ctypes.c_int, 
        ctypes.c_int
    ]

    def cuda_matmul(a_tensor, b_tensor):
        M, N = a_tensor.shape
        N_b, K = b_tensor.shape
        
        assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
        
        c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
        
        if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
        if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
        
        lib.solve(
            ctypes.c_void_p(a_tensor.data_ptr()),
            ctypes.c_void_p(b_tensor.data_ptr()),
            ctypes.c_void_p(c_tensor.data_ptr()),
            ctypes.c_int(M),
            ctypes.c_int(N),
            ctypes.c_int(K)
        )
        
        return c_tensor

    C_custom = cuda_matmul(A, B)
    print(f"{v}")
    if torch.allclose(C_custom, C_torch, atol=1e-3):
        print(f"✅ 测试通过！结果正确。")
        %timeit cuda_matmul(A, B); torch.cuda.synchronize()
    else:
        print("❌ 测试失败。结果不一致。")
        print("最大误差:", (C_custom - C_torch).abs().max().item())
    print()

正在进行矩阵乘法测试: [2048x2048] * [2048x2048]
Pytorch: 
4.8 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul.so
✅ 测试通过！结果正确。
44 ms ± 405 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v2.so
✅ 测试通过！结果正确。
21.1 ms ± 265 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v3.so
✅ 测试通过！结果正确。
21.5 ms ± 80.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v4.so
✅ 测试通过！结果正确。
6.81 ms ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v4_1.so
✅ 测试通过！结果正确。
6.91 ms ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v5.so
✅ 测试通过！结果正确。
5.31 ms ± 7.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v6.so
✅ 测试通过！结果正确。
5.37 ms ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v7.so
✅ 测试通过！结果正确。
5.49 ms ± 8.09 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

matmul_v8.so
✅ 测试通过！结果正确。
5.52 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [6]:
import torch
import ctypes
import numpy as np
import time

# --- 测试部分 ---

M, N, K = 4096, 4096, 4096

print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

device = torch.device("cuda")
A = torch.randn(M, N, device=device, dtype=torch.float32)
B = torch.randn(N, K, device=device, dtype=torch.float32)

C_torch = torch.matmul(A, B)

print("Pytorch: ")
%timeit torch.matmul(A, B); torch.cuda.synchronize()
print()

for v in sorted(os.listdir('.')):
    prefix, end = os.path.splitext(v)
    
    if end != '.so':
        continue

    lib = ctypes.CDLL(f'./{v}')

    # void solve(const float* A, const float* B, float* C, int M, int N, int K)
    # 指针对应 c_void_p, int 对应 c_int
    lib.solve.argtypes = [
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_int, 
        ctypes.c_int, 
        ctypes.c_int
    ]

    def cuda_matmul(a_tensor, b_tensor):
        M, N = a_tensor.shape
        N_b, K = b_tensor.shape
        
        assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
        
        c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
        
        if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
        if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
        
        lib.solve(
            ctypes.c_void_p(a_tensor.data_ptr()),
            ctypes.c_void_p(b_tensor.data_ptr()),
            ctypes.c_void_p(c_tensor.data_ptr()),
            ctypes.c_int(M),
            ctypes.c_int(N),
            ctypes.c_int(K)
        )
        
        return c_tensor

    C_custom = cuda_matmul(A, B)
    print(f"{v}")
    if torch.allclose(C_custom, C_torch, atol=1e-3):
        print(f"✅ 测试通过！结果正确。")
        %timeit cuda_matmul(A, B); torch.cuda.synchronize()
    else:
        print("❌ 测试失败。结果不一致。")
        print("最大误差:", (C_custom - C_torch).abs().max().item())
    print()

正在进行矩阵乘法测试: [4096x4096] * [4096x4096]
Pytorch: 
42.1 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul.so
✅ 测试通过！结果正确。
390 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

matmul_v2.so
✅ 测试通过！结果正确。
184 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v3.so
✅ 测试通过！结果正确。
181 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v4.so
✅ 测试通过！结果正确。
59.9 ms ± 374 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v4_1.so
✅ 测试通过！结果正确。
59.1 ms ± 99 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v5.so
✅ 测试通过！结果正确。
43.6 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v6.so
✅ 测试通过！结果正确。
43.4 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v7.so
✅ 测试通过！结果正确。
43.7 ms ± 76.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

matmul_v8.so
✅ 测试通过！结果正确。
43.1 ms ± 73.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [ ]:
import torch
import ctypes
import numpy as np
import time

# --- 测试部分 ---

M, N, K = 1024, 1024, 1024

print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

device = torch.device("cuda")
A = torch.randn(M, N, device=device, dtype=torch.float32)
B = torch.randn(N, K, device=device, dtype=torch.float32)

C_torch = torch.matmul(A, B)

print("Pytorch: ")
%timeit torch.matmul(A, B); torch.cuda.synchronize()
print()

v = 'matmul_v8.so'
prefix, end = os.path.splitext(v)

lib = ctypes.CDLL(f'./{v}')

# void solve(const float* A, const float* B, float* C, int M, int N, int K)
# 指针对应 c_void_p, int 对应 c_int
lib.solve.argtypes = [
    ctypes.c_void_p, 
    ctypes.c_void_p, 
    ctypes.c_void_p, 
    ctypes.c_int, 
    ctypes.c_int, 
    ctypes.c_int
]

def cuda_matmul(a_tensor, b_tensor):
    M, N = a_tensor.shape
    N_b, K = b_tensor.shape
    
    assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
    
    c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
    
    if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
    if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
    
    lib.solve(
        ctypes.c_void_p(a_tensor.data_ptr()),
        ctypes.c_void_p(b_tensor.data_ptr()),
        ctypes.c_void_p(c_tensor.data_ptr()),
        ctypes.c_int(M),
        ctypes.c_int(N),
        ctypes.c_int(K)
    )
    
    return c_tensor

C_custom = cuda_matmul(A, B)
print(f"{v}")
if torch.allclose(C_custom, C_torch, atol=1e-3):
    print(f"✅ 测试通过！结果正确。")
    %timeit cuda_matmul(A, B); torch.cuda.synchronize()
else:
    print("❌ 测试失败。结果不一致。")
    print("最大误差:", (C_custom - C_torch).abs().max().item())
print()

正在进行矩阵乘法测试: [1024x1024] * [1024x1024]
Pytorch: 
569 µs ± 3.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v8.so
✅ 测试通过！结果正确。
669 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

